In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/arcane/')
import pandas as pd

os.environ['PATH'] = '/Users/ravi.tej/anaconda3/envs/bertopicenv/bin:/Users/ravi.tej/anaconda3/condabin:/usr/bin:/bin:/usr/sbin:/sbin'

In [2]:
from src.llm_finetuning.LoraxDeployment import LoraxDeployment

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml


In [3]:
# lorax = LoraxDeployment(model_name='teknium/OpenHermes-2.5-Mistral-7B', endpoint_name='LORAX-Openhermes', instance_type='ml.g5.2xlarge')

In [4]:
lorax = LoraxDeployment(model_name='NousResearch/Nous-Capybara-34B', endpoint_name='LORAX-Capybara', instance_type='ml.g5.12xlarge')

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml


In [5]:
lorax.set_config()

In [6]:
lorax.deploy_lorax()

--------------------------------------------*

UnexpectedStatusException: Error hosting endpoint LORAX-Capybara: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html

In [7]:
df = pd.read_csv('gpt-4-responses-2024-Feb-19_cleaned.csv')

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('teknium/OpenHermes-2.5-Mistral-7B', trust_remote_code=True)

In [56]:
from botocore.config import Config
import boto3

my_config = Config(
        region_name='ap-south-1',
        retries={
            'max_attempts': 3,
            'mode': 'standard'
        },
        max_pool_connections=40  # Increase the pool size
    )

    # Create a SageMaker Runtime client with the custom configuration
sess1 = boto3.session.Session()
sagemaker_runtime = sess1.client("sagemaker-runtime", config=my_config)

In [13]:
SYSTEM_PROMPT = prompt = df.iloc[0].system_prompt

In [14]:
from src.articles._utils import truncate_text_to_token_limit

In [121]:
train_row = df.iloc[600]

In [122]:
content = truncate_text_to_token_limit(text=train_row['full_content'], encoder=tokenizer, token_limit=1536)

In [123]:
messages = [{"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": f"{content}\n"}]
context_prompt = tokenizer.decode(tokenizer.apply_chat_template(messages, add_generation_prompt=True))

In [124]:
content

'headline: FM invites JBIC for engagement with NIIF, EXIM Bank of India. \n content: Finance Minister Nirmala Sitharaman on Wednesday invited Japan Bank for International Cooperation (JBIC) for an active engagement with National Investment and Infrastructure Fund (NIIF), EXIM Bank of India and other financial institutions citing huge opportunity for investment in the country. Share Market Live NSE During the meeting with JBIC Governor Hayashi Nobumitsu here, both exchanged views on JBIC’s recent activities in India contributing towards Japan’s investment target of five trillion yen over the next five years. "The Union Finance Minister observed that India has been ranked by the JBIC as one of the most attractive #investment destinations in recent years’ survey of Japanese manufacturing companies and encouraged engagement of JBIC with Indian financial institutions such as Exim Bank @IndiaEximBank, NIIF and lending banks to leverage the strength and complementarities of both countries," t

In [125]:
parameters = {
    "do_sample": True,
    "top_p": 0.95,
    "temperature": 0.7,
    "max_new_tokens": 1024,
    "repetition_penalty": 1.02,
    "stop": ["###", "</s>", '<|im_end|>']
}
params = {"inputs": context_prompt, "parameters": parameters, "stream": False}

In [126]:
params['parameters']['adapter_id'] = 'lorax/OpenHermes-2.5-Adapter-Attributes-WatermelonUvulaMarigold'
params['parameters']['adapter_source'] = 's3'
# params['parameters']['temperature'] = 0.2
# params['parameters']['top_p'] = 0.7

In [127]:
import json
response = sagemaker_runtime.invoke_endpoint(
            EndpointName='LORAX-Openhermes',
            Body=json.dumps(params),
            ContentType="application/json")

In [130]:
r = response['Body'].read()

k = json.loads(json.loads(json.loads(r)[0]['generated_text']))

In [132]:
from src.llm_finetuning.ResponseCleanerService import ResponseCleanerService

In [137]:
ResponseCleanerService.convert_to_dicts(k['improved_summary'])

[{'emoji': '📈',
  'label': 'Investment Opportunity',
  'point': 'FM invites JBIC for investment in India; cites NIIF, EXIM Bank of India as potential partners.'},
 {'emoji': '🌍',
  'label': 'Global Collaboration',
  'point': "Discusses JBIC's recent activities in India and Japan's investment target of five trillion yen over five years."},
 {'emoji': '💡',
  'label': 'Sectors of Interest',
  'point': 'FM highlights potential investment sectors and government initiatives.'},
 {'emoji': '🎯',
  'label': "JBIC's View",
  'point': "JBIC appreciates India's initiatives and looks forward to further collaborations."}]

"!📈! Investment Opportunity: FM invites JBIC for investment in India; cites NIIF, EXIM Bank of India as potential partners. \n!🌍! Global Collaboration: Discusses JBIC's recent activities in India and Japan's investment target of five trillion yen over five years. \n!💡! Sectors of Interest: FM highlights potential investment sectors and government initiatives. \n!🎯! JBIC's View: JBIC appreciates India's initiatives and looks forward to further collaborations."